In [1]:
% load_ext autoreload
% autoreload 2

In [7]:
import utils
import train
import cvae

In [8]:
train_iter, val_iter, test, DE, EN = utils.torchtext_extract()

In [12]:
model_name = "test"

gpu = True

num_layers = 2
embed_size = 100
hidden_size = 100
latent_size = 100

num_epochs=10

anneal = utils.kl_anneal_linear

eos = EN.vocab.stoi["</s>"]
bos = EN.vocab.stoi["<s>"]
pad = EN.vocab.stoi["<pad>"]

filter_beam = [pad, bos]

In [13]:
model = cvae.CVAE(len(DE.vocab), len(EN.vocab), embed_size, hidden_size, latent_size, num_layers)
if gpu:
    model.cuda()

/home/artidoro/.local/lib/python3.5/site-packages/torch/backends/cudnn/__init__.py:84: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "


In [14]:
train.train(model,model_name, train_iter, val_iter, DE, EN, anneal, num_epochs, gpu)


  0%|          | 0/3722 [00:00<?, ?it/s]/home/artidoro/.local/lib/python3.5/site-packages/torch/backends/cudnn/__init__.py:84: UserWarning: PyTorch was compiled without cuDNN support. To use cuDNN, rebuild PyTorch making sure the library is visible to the build system.
  "PyTorch was compiled without cuDNN support. To use cuDNN, rebuild "

 77%|███████▋  | 2861/3722 [04:00<01:12, 11.91it/s]A
Exception in thread Thread-21:
Traceback (most recent call last):
  File "/usr/lib/python3.5/threading.py", line 914, in _bootstrap_inner
    self.run()
  File "/usr/local/lib/python3.5/dist-packages/tqdm/_tqdm.py", line 144, in run
    for instance in self.tqdm_cls._instances:
  File "/usr/lib/python3.5/_weakrefset.py", line 60, in __iter__
    for itemref in self.data:
RuntimeError: Set changed size during iteration

  0%|          | 0/18 [00:00<?, ?it/s]/usr/local/lib/python3.5/dist-packages/torchtext/data/field.py:321: UserWarning: volatile was removed and now has no effect. Use `with torch.no

Epoch: 1
	VPB: 3633294462875470694976143977343966328630745623371236435415267099998078883635780947449886825545394647424709337046093125638477267000027091501870338978428747776.0000 VNELBO: 360.4934 VRE: 3.5828 TKL: 356.9106
	TPB: 250084618651873548511334945634931744296446569770239460015068578680429814893687720694379619737590244452804482529460859739045888.0000 TNELBO: 288.7398 TRE: 4.3760 VKL: 284.3638


  0%|          | 1/3722 [00:00<11:51,  5.23it/s]

Epoch: 2
	VPB: 37.3938 VNELBO: 3.6215 VRE: 3.3200 TKL: 0.3015
	TPB: 79.1515 TNELBO: 4.3714 TRE: 3.6399 VKL: 0.7314


  0%|          | 0/3722 [00:00<?, ?it/s]

...

Epoch: 8
	VPB: 17.4759 VNELBO: 2.8608 VRE: 2.8487 TKL: 0.0121
	TPB: 12.8057 TNELBO: 2.5499 TRE: 2.5184 VKL: 0.0315


  0%|          | 0/3722 [00:00<?, ?it/s]

Epoch: 9
	VPB: 23.5827 VNELBO: 3.1605 VRE: 2.8453 TKL: 0.3152
	TPB: 11.9269 TNELBO: 2.4788 TRE: 2.4458 VKL: 0.0330


100%|██████████| 18/18 [00:00<00:00, 41.82it/s]

Epoch: 10
	VPB: 21.1342 VNELBO: 3.0509 VRE: 2.8447 TKL: 0.2062
	TPB: 11.1089 TNELBO: 2.4077 TRE: 2.3836 VKL: 0.0241


In [ ]:
val_perp = utils.eval_vae(model, val_iter, pad, gpu=gpu)
print(val_perp)

In [ ]:
EN.vocab.stoi['</s>']